In [ ]:
import pandas as pd
from pathlib import Path


root_dir = Path.cwd().parent
target_data = root_dir / "datas" / "수능 국어.csv"
df = pd.read_csv(target_data, encoding="utf-8")
df

In [ ]:
from datasets import Dataset

# 데이터 변환: 쿼리 생성 및 정답 포맷팅
data_list = []

for idx, row in df.iterrows():
    passage = row['passage']
    problem = row['problem']
    choice_1 = row['choice_1']
    choice_2 = row['choice_2']
    choice_3 = row['choice_3']
    choice_4 = row['choice_4']
    choice_5 = row['choice_5']
    answer = str(int(row['answer']))

    query = f"""다음을 읽고 물음에 답하시오
<지문>
{passage}
</지문>
<문제>
{problem}
1. {choice_1}
2. {choice_2}
3. {choice_3}
4. {choice_4}
5. {choice_5}
</문제>

<think> 태그 안에 추론 과정을 작성하고, <answer> 태그 안에 정답의 숫자만 적으시오. 이 외의 답은 모두 오답으로 처리됩니다."""

    data_list.append({'query': query, 'answer': answer})

# 학습/평가 데이터 분할 (7:3)
split_idx = len(data_list) // 10 * 7

training_data = data_list[:split_idx]
testing_data = data_list[split_idx:]

# Datasets 형태로 변환 및 저장
train_dataset = Dataset.from_list(training_data)
test_dataset = Dataset.from_list(testing_data)

# 디스크에 저장 (빠른 로딩을 위해)
train_dataset.save_to_disk("./training_dataset")
test_dataset.save_to_disk("./testing_dataset")

print(f"✅ 학습 데이터: {len(train_dataset)} 샘플 저장 완료")
print(f"✅ 평가 데이터: {len(test_dataset)} 샘플 저장 완료")